In [39]:
import pandas as pd # veri analizi ve veri işleme için kullanılır (dataFrame yapısı sağlar)
from sklearn.feature_extraction.text import CountVectorizer # metinleri sayısal forma dönüştürür (BoW yöntemi)
import re #metinlerde temizleme işlemi yapar
from collections import Counter # elemanların frekansını sağlar (ör: kelime sıklığı hesaplama)

In [40]:
df= pd.read_csv("IMDB_Dataset.csv")

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [42]:
df.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [43]:
#metin verilerini al
documents= df["review"]
labels= df["sentiment"] #positive veya negatif

In [47]:
#metin temizleme
def clean_text(text):
  text=text.lower() #büyük küçük harf çevirme
  text=re.sub(r"\d+", "",text) #rakamları temizleme
  text=re.sub(r"[^\w\s]", "", text) #özel karakterlerin kaldırılması
  text= " ".join([word for word in text.split() if len(word) >2]) #kısa kelimelerin temizlenmesi
  return text #temizlenmiş text'i return et

#metinleri temizle
cleaned_doc=[clean_text(row) for row in documents]

In [45]:
#metin temizleme 2
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def clean_text(text):
  text=text.lower() #büyük küçük harf çevirme
  text=re.sub(r"\d+", "",text) #rakamları temizleme
  text=re.sub(r"[^\w\s]", "", text) #özel karakterlerin kaldırılması
  text=BeautifulSoup(text,"html.parser").get_text() #html etiketlerini temizler
  text = re.sub(r"http\S+", "",text,flags=re.MULTILINE) # URL'leri kalırmak için yapılacak işlemler
  stop_words= set(stopwords.words('english'))
  text= " ".join([word for word in text.split() if len(word) >2 and word not in stop_words]) #kısa kelimelerin temizlenmesi

cleaned_doc=[clean_text(row) for row in documents]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [48]:
# Bag of Words
vectorizer=CountVectorizer() # vectorizer tanımlama

X = vectorizer.fit_transform(cleaned_doc[:100]) #metin -> sayısal hale getir / temizlenmiş verilerden ilk 100'ünü göster

feature_names = vectorizer.get_feature_names_out() #kelime kümesini göster

vektor_temsili = X.toarray() #vektör temsilini göster
print("Vektör temsili: ", vektor_temsili)

df_bow= pd.DataFrame(vektor_temsili, columns = feature_names) #sütun adları kelimeler olacak şekilde bag of words tablosu oluşturur
df_bow.head(10)

Vektör temsili:  [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


,abbot,abbreviated,abetted,abiding,ability,able,about,aboveaverage,abraham,abrahams,...,zany,zellweger,zerog,zeus,zombie,zombiebr,zone,zoo,zooms,zwick
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
4,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
#kelime frekanslarını göster
word_counts= X.sum(axis=0).A1 # her kelimenin tüm belgelerdeli toplam geçiş sayısını hesaplar
word_freq=dict(zip(feature_names, word_counts))
print("Word Frequencies: ", word_freq)

#ilk 5 kelimeyi print ettir
most_common_5_words = Counter(word_freq).most_common(5)
print("Most Common 5 words:", most_common_5_words)

Word Frequencies:  {'abbot': np.int64(1), 'abbreviated': np.int64(1), 'abetted': np.int64(2), 'abiding': np.int64(2), 'ability': np.int64(1), 'able': np.int64(2), 'about': np.int64(71), 'aboveaverage': np.int64(1), 'abraham': np.int64(1), 'abrahams': np.int64(1), 'absolute': np.int64(1), 'absolutely': np.int64(7), 'absorb': np.int64(1), 'abstracted': np.int64(1), 'absurd': np.int64(2), 'abusive': np.int64(1), 'academy': np.int64(1), 'accent': np.int64(2), 'accented': np.int64(1), 'accents': np.int64(1), 'accentuate': np.int64(1), 'accepted': np.int64(3), 'accepting': np.int64(1), 'accepts': np.int64(1), 'accident': np.int64(2), 'accidentally': np.int64(1), 'accomplish': np.int64(1), 'accomplished': np.int64(1), 'accomplishes': np.int64(1), 'according': np.int64(2), 'account': np.int64(1), 'accountant': np.int64(1), 'accounts': np.int64(1), 'accurate': np.int64(3), 'accuses': np.int64(1), 'accustomed': np.int64(1), 'achieve': np.int64(1), 'achieved': np.int64(1), 'achievement': np.int64